In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 
import numpy as np
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
from skimage import io
import os
import pandas as pd

In [2]:
file_label = open("project_data/labels/train_labels.txt", "r")
val_labels = open("project_data/labels/val_labels.txt", "r")
bho = [(line.strip()).split() for line in file_label]
numeri_test = [numero[0] for numero in bho]
y_train = [nome[1] for nome in bho]

dio = [(line.strip()).split() for line in val_labels]
numeri_val = [numero[0] for numero in dio]
y_val = [nome[1] for nome in dio]

#X_train è unsorted
#X_train contiene tutte le immagini ma non sono sortate per nome
import glob
X_train = []
X_val = []
nome_cartella = ["bacteria", "corona", "normal", "viral"]

for numero in numeri_test:
    for cartella in nome_cartella:
        for img in glob.glob("project_data/images/train/{}/{}.jpeg".format(cartella, numero)):
            n = io.imread(img, as_gray=True)
            n = np.reshape(n,65536)
            n = n.tolist()
            X_train.append(n) 
for numero in numeri_val:
    for cartella in nome_cartella:
        for img in glob.glob("project_data/images/val/{}/{}.jpeg".format(cartella, numero)):
            n = io.imread(img, as_gray=True)
            n = np.reshape(n,65536)
            n = n.tolist()
            X_val.append(n) 
print(len(X_train[0]))
#plt.imshow(X_train[0])

65536


In [3]:
import numpy as np 

X_train = np.stack(X_train, axis = 0)

X_train.shape
#plt.imshow(X_train[0])

(3569, 65536)

In [4]:
X_train.min(), X_train.max()
#plt.imshow(X_train[0])

(0.0, 1.0)

In [5]:
y_train = pd.factorize(y_train) # per rendere il vettore da 0 a 3
y_train = y_train[0]
y_val = pd.factorize(y_val)
y_val = y_val[0]

In [6]:
y_train

array([0, 1, 0, ..., 1, 0, 0])

In [7]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes = 4)
y_val = to_categorical(y_val, num_classes = 4)

Using TensorFlow backend.


In [8]:
y_train.shape
X_train.shape

(3569, 65536)

In [9]:
X_train_npy = []
X_val_npy = []
for nome in glob.glob('project_data/features/train/*'):
    n = np.load('{}'.format(nome))
    n = n.tolist()
    X_train_npy.append(n)
for nome in glob.glob('project_data/features/val/*'):
    n = np.load('{}'.format(nome))
    n = n.tolist()
    X_val_npy.append(n)
X_val_npy = np.stack(X_val_npy, axis = 0)
X_train_npy = np.stack(X_train_npy, axis = 0)


In [10]:
X_train_npy.shape

(3569, 84)

In [28]:
X_train_npy[0]

array([1.90403748, 0.        , 0.        , 0.        , 0.03189665,
       1.31642258, 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.5409174 , 2.21305847, 1.59444511, 0.        ,
       0.46223465, 0.        , 2.05552173, 0.        , 0.        ,
       2.55537438, 0.        , 0.36178216, 2.8339119 , 1.8925842 ,
       1.78515053, 0.120464  , 0.        , 0.        , 0.        ,
       0.        , 1.10024309, 0.        , 0.        , 0.        ,
       1.11137557, 0.        , 0.02469489, 1.99558008, 2.07351971,
       1.75851333, 1.52332735, 0.        , 0.        , 0.        ,
       2.16634178, 0.64096564, 0.        , 1.78030562, 0.        ,
       0.25273788, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 2.16830397, 0.        , 0.        ,
       0.52866948, 0.        , 0.48487186, 3.05217528, 0.        ,
       0.        , 0.        , 1.57993138, 0.        , 0.        ,
       0.        , 2.23386049, 0.        , 0.51694357, 0.     

In [15]:
from keras.layers import Input, Dense, Activation
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers

In [30]:
img_input = Input(shape=(84,)) #input layer, occhio che shape è una tupla

x = Dense(units = 60, activation = 'relu',  kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),)(img_input) 
x = Dense(units = 40, activation = 'relu',  kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),)(x)#img_input viene passato a x
x = Dropout(0.5)(x)
x = Dense(units = 4, activation = 'sigmoid')(x)

model = Model(inputs = img_input, output = x)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2, restore_best_weights=True)
model.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])
model.fit(X_train_npy, y_train, batch_size = 20, 
          epochs = 50, validation_split = 0.2, callbacks=[callback], shuffle = True)

/home/marcusvuk/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


Train on 2855 samples, validate on 714 samples
Epoch 1/50
2855/2855 [==============================] - 0s 128us/step - loss: 0.7397 - accuracy: 0.7380 - val_loss: 0.3849 - val_accuracy: 0.8473
Epoch 2/50
2855/2855 [==============================] - 0s 60us/step - loss: 0.4524 - accuracy: 0.8308 - val_loss: 0.3603 - val_accuracy: 0.8515
Epoch 3/50
2855/2855 [==============================] - 0s 49us/step - loss: 0.4118 - accuracy: 0.8497 - val_loss: 0.3553 - val_accuracy: 0.8515
Epoch 4/50
2855/2855 [==============================] - 0s 63us/step - loss: 0.3989 - accuracy: 0.8574 - val_loss: 0.3473 - val_accuracy: 0.8515
Epoch 5/50
2855/2855 [==============================] - 0s 75us/step - loss: 0.3805 - accuracy: 0.8623 - val_loss: 0.3621 - val_accuracy: 0.8585
Epoch 6/50
2855/2855 [==============================] - 0s 50us/step - loss: 0.3684 - accuracy: 0.8690 - val_loss: 0.3439 - val_accuracy: 0.8557
Epoch 7/50
2855/2855 [==============================] - 0s 49us/step - loss: 0.354

In [31]:
print(model.metrics_names)
model.evaluate(X_val_npy, y_val)

['loss', 'accuracy']
1189/1189 [==============================] - 0s 24us/step


[0.8465778596567647, 0.7830109596252441]

In [24]:
y_pred = model.predict(X_val_npy)

In [25]:
y_pred

array([[8.8745201e-01, 1.4901161e-07, 0.0000000e+00, 0.0000000e+00],
       [9.9997675e-01, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [2.9802322e-08, 9.0317649e-01, 4.0809005e-02, 0.0000000e+00],
       ...,
       [7.7486038e-07, 8.4098244e-01, 2.8810799e-03, 5.9604645e-08],
       [1.6093254e-06, 7.6544940e-01, 5.0687939e-02, 2.3841858e-07],
       [7.2065141e-06, 6.3802153e-01, 4.5401886e-02, 1.8005805e-07]],
      dtype=float32)